In [1]:
import numpy as np 
import pandas as pd
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import seaborn as sns
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

from sklearn.tree import DecisionTreeClassifier
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss  
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from xgboost import XGBClassifier
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')

## **Four Datasets**
1) Default Credit Card Clients - Classification Problem

2) Parkinson Featutres -  Classification Problem

3) Mice Protein - Regresression Problem

4) Super Conduct - Regression Problem


In [2]:
credit_card_ds = "https://raw.githubusercontent.com/Rumsha001/ML2/main/dataset/default_credit_card.csv"
cc_df = pd.read_csv(credit_card_ds)
cc_df.dropna(axis=0,inplace=True)
cc_df

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000,1,3,1,39,0,0,0,0,0,0,188948,192815,208365,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,29997,150000,1,3,2,43,-1,-1,-1,-1,0,0,1683,1828,3502,8979,5190,0,1837,3526,8998,129,0,0,0
29997,29998,30000,1,2,2,37,4,3,2,-1,0,0,3565,3356,2758,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,29999,80000,1,3,1,41,1,-1,0,0,0,-1,-1645,78379,76304,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [3]:
parkinson_ds = "https://raw.githubusercontent.com/Rumsha001/ML2/main/dataset/pd_speech_features.csv"
parkinson_df = pd.read_csv(parkinson_ds)
parkinson_df.dropna(axis=0,inplace=True)
parkinson_df

,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,rapJitter,ppq5Jitter,ddpJitter,locShimmer,locDbShimmer,apq3Shimmer,apq5Shimmer,apq11Shimmer,ddaShimmer,meanAutoCorrHarmonicity,meanNoiseToHarmHarmonicity,meanHarmToNoiseHarmonicity,minIntensity,maxIntensity,meanIntensity,f1,f2,f3,f4,b1,b2,b3,b4,GQ_prc5_95,GQ_std_cycle_open,GQ_std_cycle_closed,GNE_mean,GNE_std,GNE_SNR_TKEO,...,tqwt_skewnessValue_dec_34,tqwt_skewnessValue_dec_35,tqwt_skewnessValue_dec_36,tqwt_kurtosisValue_dec_1,tqwt_kurtosisValue_dec_2,tqwt_kurtosisValue_dec_3,tqwt_kurtosisValue_dec_4,tqwt_kurtosisValue_dec_5,tqwt_kurtosisValue_dec_6,tqwt_kurtosisValue_dec_7,tqwt_kurtosisValue_dec_8,tqwt_kurtosisValue_dec_9,tqwt_kurtosisValue_dec_10,tqwt_kurtosisValue_dec_11,tqwt_kurtosisValue_dec_12,tqwt_kurtosisValue_dec_13,tqwt_kurtosisValue_dec_14,tqwt_kurtosisValue_dec_15,tqwt_kurtosisValue_dec_16,tqwt_kurtosisValue_dec_17,tqwt_kurtosisValue_dec_18,tqwt_kurtosisValue_dec_19,tqwt_kurtosisValue_dec_20,tqwt_kurtosisValue_dec_21,tqwt_kurtosisValue_dec_22,tqwt_kurtosisValue_dec_23,tqwt_kurtosisValue_dec_24,tqwt_kurtosisValue_dec_25,tqwt_kurtosisValue_dec_26,tqwt_kurtosisValue_dec_27,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,0.000018,0.00067,0.00129,0.00200,0.05883,0.517,0.03011,0.03496,0.04828,0.09034,0.970805,0.036223,18.995,69.997496,76.088046,72.465512,539.342735,1031.849040,2447.162183,3655.054806,101.092218,83.147440,255.214830,396.643631,0.77778,11.7245,2.8277,1.17300,0.265120,0.083127,...,0.071728,0.010352,-2.730300,66.5007,36.9934,26.3508,51.9577,21.5451,8.2488,8.0024,6.9635,6.9189,5.0622,4.6068,7.7218,2.7224,2.4171,2.9383,4.2077,3.1541,2.8531,2.7496,2.1550,2.9457,2.1993,1.9830,1.8314,2.0062,1.6058,1.5466,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,0.000016,0.00052,0.00112,0.00157,0.05516,0.502,0.02320,0.03675,0.06195,0.06961,0.984322,0.017974,21.497,67.415903,73.046374,71.528945,564.363614,1016.367294,2383.565201,3498.681572,58.465428,86.487292,248.357127,218.229722,0.81250,13.8284,2.8908,1.02210,0.220040,0.127410,...,0.729330,0.780410,5.229400,8643.9860,3962.0554,2976.2411,4329.0607,4005.1329,1127.6762,116.5331,20.2332,13.6395,12.5514,13.6965,60.8974,6.2958,3.0323,3.4039,4.3794,2.4329,2.0585,2.1839,2.2061,3.0999,1.9824,1.6227,1.5783,2.0470,1.5772,1.5530,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,0.000015,0.00057,0.00111,0.00171,0.09902,0.897,0.05094,0.06497,0.07772,0.15282,0.974846,0.026313,17.651,62.661706,71.633549,68.086583,548.444604,1032.406341,2357.826954,3678.128717,160.387771,54.685168,151.694847,84.240339,0.81818,26.9273,2.6975,0.84951,0.157560,0.116890,...,0.269510,-0.005522,0.350540,29.2717,32.4971,38.9453,85.1480,62.5132,9.7308,6.8890,6.1357,5.4926,3.9078,4.0864,6.9828,3.4411,3.5173,3.7204,4.2212,3.4881,3.4851,3.3007,2.0427,3.1436,2.1203,1.6627,1.6731,3.2597,1.5921,1.5399,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,0.000046,0.00149,0.00268,0.00446,0.05451,0.527,0.02395,0.02857,0.04462,0.07185,0.968343,0.042003,19.865,76.306989,81.000749,79.190593,819.529588,1201.813897,3154.035654,4122.163933,238.667052,191.984916,573.752909,526.147599,0.98548,139.5744,1.6961,0.83405,0.172950,0.147370,...,0.366920,-0.492650,0.191640,591.0116,65.3827,53.9852,45.4458,34.6650,88.3259,46.6869,23.2911,19.2022,42.3957,100.3649,7.1967,6.3892,6.5496,6.0264,4.7656,4.8909,4.2531,3.0295,2.0362,1.8478,2.5776,2.2064,1.9491,1.9120,1.8829,6.9761,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4

In [4]:
mice_ds = "https://raw.githubusercontent.com/Rumsha001/ML2/main/dataset/mice_protein_regression.csv"
mice_df = pd.read_csv(mice_ds)
mice_df.dropna(axis=0,inplace=True)
mice_df

,MouseID,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,pERK_N,pJNK_N,PKCA_N,pMEK_N,pNR1_N,pNR2A_N,pNR2B_N,pPKCAB_N,pRSK_N,AKT_N,BRAF_N,CAMKII_N,CREB_N,ELK_N,ERK_N,GSK3B_N,JNK_N,MEK_N,TRKA_N,RSK_N,APP_N,Bcatenin_N,SOD1_N,MTOR_N,P38_N,pMTOR_N,DSCR1_N,AMPKA_N,NR2B_N,...,TIAM1_N,pP70S6_N,NUMB_N,P70S6_N,pGSK3B_N,pPKCG_N,CDK5_N,S6_N,ADARB1_N,AcetylH3K9_N,RRP1_N,BAX_N,ARC_N,ERBB4_N,nNOS_N,Tau_N,GFAP_N,GluR3_N,GluR4_N,IL1B_N,P3525_N,pCASP9_N,PSD95_N,SNCA_N,Ubiquitin_N,pGSK3B_Tyr216_N,SHH_N,BAD_N,BCL2_N,pS6_N,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,Genotype,Treatment,Behavior,class
75,3415_1,0.649781,0.828696,0.405862,2.921435,5.167979,0.207174,0.176640,3.728084,0.239283,1.666579,0.969466,0.321260,0.406562,0.255293,1.056343,1.099300,1.925284,2.231671,0.426859,0.873228,0.512773,0.343220,0.167629,1.852756,3.366492,1.503675,0.246019,0.305424,0.851006,0.165179,0.441557,2.578215,0.340857,0.487664,0.401050,0.781365,0.534033,0.431846,0.618023,...,0.427297,0.252056,0.242086,1.061181,0.163758,1.862368,0.321175,0.541041,2.159547,0.134639,0.158178,0.196124,0.132001,0.161120,0.210988,0.210379,0.128145,0.292461,0.152597,0.527242,0.348062,1.687551,2.436587,0.168172,1.139154,0.987774,0.239752,0.139052,0.112926,0.132001,0.129363,0.486912,0.125152,0.146865,0.143517,1.627181,Control,Memantine,C/S,c-CS-m
76,3415_2,0.616481,0.841974,0.388584,2.862575,5.194163,0.223433,0.167725,3.648240,0.221030,1.565150,0.991845,0.342918,0.396910,0.270815,1.032790,1.078712,1.893047,2.330987,0.409185,0.789013,0.511502,0.344807,0.166695,1.852532,3.431159,1.471674,0.242060,0.300172,0.833648,0.162661,0.437253,2.538026,0.337682,0.472618,0.402318,0.786524,0.533219,0.429871,0.603176,...,0.425837,0.252017,0.242120,1.063985,0.169981,1.692204,0.320473,0.539226,2.167098,0.136273,0.161579,0.193348,0.139008,0.161529,0.210798,0.193398,0.134583,0.291489,0.151884,0.519787,0.365119,1.662872,2.354728,0.174704,1.227851,0.981655,0.249031,0.133787,0.121607,0.139008,0.143084,0.467833,0.112857,0.161132,0.145719,1.562096,Control,Memantine,C/S,c-CS-m
77,3415_3,0.637424,0.852882,0.400561,2.968155,5.350820,0.208790,0.173261,3.814545,0.222300,1.741732,0.989999,0.328976,0.397930,0.263795,1.092289,1.094043,1.915958,1.952540,0.454075,0.793754,0.533205,0.352048,0.172559,1.965786,3.434249,1.541802,0.251952,0.345908,0.881393,0.165278,0.443460,2.609615,0.342398,0.493903,0.410913,0.812791,0.564699,0.441179,0.640232,...,0.445214,0.252303,0.241042,1.059453,0.175230,1.654840,0.317856,0.561875,2.182447,0.135914,0.167508,0.192238,0.134804,0.181034,0.202534,0.204300,0.129555,0.270062,0.153629,0.506309,0.349753,1.716362,2.437317,0.156001,1.226103,0.994146,0.247956,0.142324,0.130261,0.134804,0.147673,0.462501,0.116433,0.160594,0.142879,1.571868,Control,Memantine,C/S,c-CS-m
78,3415_4,0.576815,0.755390,0.348346,2.624901,4.727509,0.205892,0.161192,3.778530,0.194153,1.505475,0.819167,0.311548,0.385822,0.241562,0.948414,0.924822,1.755842,2.297212,0.484818,0.742183,0.459194,0.337510,0.161982,1.364714,2.814990,1.440795,0.234902,0.274975,0.766452,0.150581,0.407608,2.311999,0.312112,0.466305,0.394740,0.740603,0.522971,0.391918,0.585055,...,0.399932,0.277458,0.208718,0.969007,0.153779,2.027937,0.299825,0.466762,1.795148,0.126091,0.144363,0.176977,0.118982,0.163632,0.195311,0.194126,0.119356,0.276939,0.134635,0.526503,0.316600,1.586368,2.382140,0.145049,1.056498,0.970067,0.233225,0.133637,0.107321,0.118982,0.121290,0.479110,0.102831,0.144238,0.141681,1.646608,Control,Memantine,C/S,c-CS-m
79,3415_5,0.542545,0.757917,0.350051,2.634509,4.735602,0.210526,0.165671,3.871971,0.194297,1.531613,0.815057,0.315001,0.391525,0.254142,0.955370,0.938465,1.744844,2.477854,0.476502,0.715880,0.486307,0.338668,0.160600,1.398174,2.891018,1.469402,0.234870,0.270033,0.755438,0.149555,0.411360,2.320861,0.309478,0.472332,0.393666,0.739209,0.542207,0.389947,0.587400,...,0.405838,0.354671,0.212714,0.972906,0.163062,1.969799,0.303008,0.491797,1.763485,0.124348,0.153741,0.188106,0.128635,0.167972,0.196992,0.197117,0

In [5]:
super_conduct_ds = "https://raw.githubusercontent.com/Rumsha001/ML2/main/dataset/super_conduct_regression.csv"
sc_df = pd.read_csv(super_conduct_ds)
sc_df.dropna(axis=0,inplace=True)
sc_df

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,wtd_std_atomic_mass,mean_fie,wtd_mean_fie,gmean_fie,wtd_gmean_fie,entropy_fie,wtd_entropy_fie,range_fie,wtd_range_fie,std_fie,wtd_std_fie,mean_atomic_radius,wtd_mean_atomic_radius,gmean_atomic_radius,wtd_gmean_atomic_radius,entropy_atomic_radius,wtd_entropy_atomic_radius,range_atomic_radius,wtd_range_atomic_radius,std_atomic_radius,wtd_std_atomic_radius,mean_Density,wtd_mean_Density,gmean_Density,wtd_gmean_Density,entropy_Density,wtd_entropy_Density,range_Density,wtd_range_Density,std_Density,...,wtd_mean_ElectronAffinity,gmean_ElectronAffinity,wtd_gmean_ElectronAffinity,entropy_ElectronAffinity,wtd_entropy_ElectronAffinity,range_ElectronAffinity,wtd_range_ElectronAffinity,std_ElectronAffinity,wtd_std_ElectronAffinity,mean_FusionHeat,wtd_mean_FusionHeat,gmean_FusionHeat,wtd_gmean_FusionHeat,entropy_FusionHeat,wtd_entropy_FusionHeat,range_FusionHeat,wtd_range_FusionHeat,std_FusionHeat,wtd_std_FusionHeat,mean_ThermalConductivity,wtd_mean_ThermalConductivity,gmean_ThermalConductivity,wtd_gmean_ThermalConductivity,entropy_ThermalConductivity,wtd_entropy_ThermalConductivity,range_ThermalConductivity,wtd_range_ThermalConductivity,std_ThermalConductivity,wtd_std_ThermalConductivity,mean_Valence,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
0,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,53.622535,775.425000,1010.268571,718.152900,938.016780,1.305967,0.791488,810.6,735.985714,323.811808,355.562967,160.250000,105.514286,136.126003,84.528423,1.259244,1.207040,205,42.914286,75.237541,69.235569,4654.35725,2961.502286,724.953211,53.543811,1.033129,0.814598,8958.571,1579.583429,3306.162897,...,111.727143,60.123179,99.414682,1.159687,0.787382,127.05,80.987143,51.433712,42.558396,6.905500,3.846857,3.479475,1.040986,1.088575,0.994998,12.878,1.744571,4.599064,4.666920,107.756645,61.015189,7.062488,0.621979,0.308148,0.262848,399.97342,57.127669,168.854244,138.517163,2.25,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059,29.00
1,5,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,47.094633,53.979870,766.440000,1010.612857,720.605511,938.745413,1.544145,0.807078,810.6,743.164286,290.183029,354.963511,161.200000,104.971429,141.465215,84.370167,1.508328,1.204115,205,50.571429,67.321319,68.008817,5821.48580,3021.016571,1237.095080,54.095718,1.314442,0.914802,10488.571,1667.383429,3767.403176,...,112.316429,69.833315,101.166398,1.427997,0.838666,127.05,81.207857,49.438167,41.667621,7.784400,3.796857,4.403790,1.035251,1.374977,1.073094,12.878,1.595714,4.473363,4.603000,172.205316,61.372331,16.064228,0.619735,0.847404,0.567706,429.97342,51.413383,198.554600,139.630922,2.00,2.257143,1.888175,2.210679,1.557113,1.047221,2,1.128571,0.632456,0.468606,26.00
2,4,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,51.968828,53.656268,775.425000,1010.820000,718.152900,939.009036,1.305967,0.773620,810.6,743.164286,323.811808,354.804183,160.250000,104.685714,136.126003,84.214573,1.259244,1.132547,205,49.314286,75.237541,67.797712,4654.35725,2999.159429,724.953211,53.974022,1.033129,0.760305,8958.571,1667.383429,3306.162897,...,112.213571,60.123179,101.082152,1.159687,0.786007,127.05,81.207857,51.433712,41.639878,6.905500,3.822571,3.479475,1.037439,1.088575,0.927479,12.878,1.757143,4.599064,4.649635,107.756645,60.943760,7.062488,0.619095,0.308148,0.250477,399.97342,57.127669,168.854244,138.540613,2.25,2.271429,2.213364,2.232679,1.368922,1.029175,1,1.114286,0.433013,0.444697,19.00
3,4,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.768010,51.968828,53.639405,775.425000,1010.544286,718.152900,938.512777,1.305967,0.78320

In [6]:
#dropping IDs
cc_df.drop(columns="ID", inplace=True)
parkinson_df.drop(columns="id", inplace=True)
mice_df.drop(columns=["MouseID","class"], inplace=True)

#replacing categorical values to binary
mice_df.Genotype[mice_df.Genotype=='Control']=1
mice_df.Genotype[mice_df.Genotype=='Ts65Dn']=0
mice_df.Treatment[mice_df.Treatment=='Memantine']=1
mice_df.Treatment[mice_df.Treatment=='Saline']=0
mice_df.Behavior[mice_df.Behavior=='C/S']=1
mice_df.Behavior[mice_df.Behavior=='S/C']=0


## **Dimensionality Reduction Methods**

In [7]:
ALGORITHMS = {
    "XGBOOST":"XGBOOST",
    "RANDOM_FOREST":"RANDOM_FOREST",
    "LINEAR_REGRESSION":"LINEAR_REGRESSION",
    "VOTING_REGRESSION":"VOTING_REGRESSION",
}

DR = {
    "PCA":"PCA",
    "LDA":"LDA",
    "L1":"L1",
    "RF":"RF",
    "XG":"XG",
    "RFE":"RFE"
}
        

In [8]:
#this function is implmening DecisionreeClassifier for RFE
def recursive_fs(data,X_train_non_scaled,n,type):
   X_train = data["X_train"]
   X_test = data["X_test"]
   y_train = data["y_train"]
   y_test = data["y_test"]
   estimator = DecisionTreeClassifier()
   selector = RFE(estimator, n_features_to_select=n, step=1)
   selector = selector.fit(X_train, y_train.values.ravel())
   selected_feat = X_train_non_scaled.columns[(selector.get_support())]
   print(f"Features Selected By {type}: {selected_feat}")
   return selector.transform(X_test), selector.transform(X_train)

#######################################################################################################################3
#This function is used select most important feature from the Dataset provided the classification model.
def feature_selection(data,X_train_non_scaled,model,type):
  X_train = data["X_train"]
  X_test = data["X_test"]
  y_train = data["y_train"]
  y_test = data["y_test"]
  sel_ = SelectFromModel(model)
  sel_.fit(X_train, y_train.values.ravel())
  selected_feat = X_train_non_scaled.columns[(sel_.get_support())]
  print(f"Features Selected By {type}: {selected_feat}")
  return sel_.transform(X_test), sel_.transform(X_train)

###############################################################################################################
#This function fits different models on the Dataset. It checks what algrithm to select based on the Dataset Object provided as a parameter.
def compare_algorithms(X_train,Y_train,X_test,Y_test,method,dataset_object):
    reduction_type = method
    algos = dataset_object['ALGORITHMS'];
    data_prediction_type = dataset_object["TYPE"]

    for algo in algos:
      if algo == ALGORITHMS["RANDOM_FOREST"] and data_prediction_type == "Classification":
          kfold = model_selection.KFold(n_splits=10, random_state=7, shuffle=True)
          model = RandomForestClassifier(max_depth = 2, random_state = 0)
          result_auc = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring='roc_auc')
          result_accuracy = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
          result_precision = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring='precision')
          print(f"Results for Type = {reduction_type} | Model = {algo}")
          print(f"Accuracy: {result_accuracy.mean()}")
          print(f"Precision: {result_precision.mean()}")
          print(f"ROC AUC: {result_auc.mean()}")
          print(100*"=")

      if algo == ALGORITHMS["XGBOOST"] and data_prediction_type == "Classification":
          kfold = model_selection.KFold(n_splits=10, random_state=7, shuffle=True)
          model = XGBClassifier()
          result_auc = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring='roc_auc')
          result_accuracy = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
          result_precision = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring='precision')
          print(f"Results for Type = {reduction_type} | Model = {algo}")
          print(f"Accuracy: {result_accuracy.mean()}")
          print(f"Precision: {result_precision.mean()}")
          print(f"ROC AUC: {result_auc.mean()}")
          print(100*"=")

     
      if algo == ALGORITHMS["LINEAR_REGRESSION"] and data_prediction_type == "Regression":
          print(f"Results of:{algo}")
          model = LinearRegression()
          model.fit(X_train, Y_train)
          y_pred = model.predict(X_test)
          print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, y_pred))
          print('Mean Squared Error:', metrics.mean_squared_error(Y_test, y_pred))
          print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, y_pred)))
          print(100*"=")

      if algo == ALGORITHMS["VOTING_REGRESSION"] and data_prediction_type == "Regression":
          print(f"Results of:{algo}")
          regressor1 = LinearRegression()
          regressor2 = RandomForestRegressor(n_estimators=100, random_state=1)
          model = VotingRegressor([('lr', regressor1), ('rf', regressor2)])
          model.fit(X_train,Y_train)
          y_pred = model.predict(X_test)
          print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, y_pred))
          print('Mean Squared Error:', metrics.mean_squared_error(Y_test, y_pred))
          print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, y_pred)))
          print(100*"=")
  
###########################################################################################################

def select_n_components(var_ratio, goal_var: float) -> int:
    # Set initial variance explained so far
    total_variance = 0.0
    n_components = 0
    # For the explained variance of each feature:
    for explained_variance in var_ratio:
        # Add the explained variance to the total
        total_variance += explained_variance
        n_components += 1
        if total_variance >= goal_var:
            break
            
    # Return the number of components
    return n_components

#####################################################################################################

def get_n_components(data):
   print("LDA:Getting Components")
   lda = LDA(n_components=None)
   X_train = lda.fit(data["X_train"],data['y_train'].values.ravel())
   lda_var_ratios = lda.explained_variance_ratio_
   n = select_n_components(lda_var_ratios, 0.90)
   print(f"Best Number Of Components For LDA: {n}")
   return n

# Main Function  
def start_comparison(dataset_object):
  df = dataset_object["DATAFRAME"]
  data_prediction_type = dataset_object["TYPE"]
  X = df.iloc[:, :-1]
  y = df.iloc[: , -1:]
  X_train_non_scaled, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.5, random_state= 0)
  sc = StandardScaler()
  X_train = sc.fit_transform(X_train_non_scaled)
  X_test = sc.transform(X_test)
  data = {
        "X_train":X_train, 
        "X_test":X_test, 
        "y_train":y_train, 
        "y_test":y_test
    }
  print(f"Training Data Shape Before Applying FS or DR {X_train.shape}")

  for method in DR:
    if method == DR["PCA"]:
      reduction_type = "PCA"
      pca = PCA(dataset_object["N_COMPONENTS"])
      X_train = pca.fit_transform(data["X_train"])
      X_test = pca.transform(data["X_test"])
      Y_train = data["y_train"]
      print(f"Training Data Shape After Applying PCA {X_train.shape}")
      compare_algorithms(X_train,Y_train.values.ravel(),X_test,y_test,method,dataset_object)

    if method == DR["LDA"] and data_prediction_type == "Classification":
      n = get_n_components(data)
      lda = LDA(n_components=n)
      X_train = lda.fit_transform(data["X_train"],data['y_train'].values.ravel())
      X_test = lda.transform(data["X_test"])
      Y_train = data["y_train"]
      compare_algorithms(X_train,Y_train.values.ravel(),X_test,y_test,method,dataset_object)

    if method == DR["L1"] and data_prediction_type == "Classification":
      X_test,X_train  = feature_selection(data,X_train_non_scaled,LogisticRegression(random_state = 0,solver = 'liblinear', multi_class= 'auto'),method)
      Y_train = data["y_train"]
      compare_algorithms(X_train,Y_train.values.ravel(),X_test,y_test,method,dataset_object)

    if method == DR["RF"] and data_prediction_type == "Classification":
      X_test,X_train  = feature_selection(data,X_train_non_scaled,RandomForestClassifier(n_estimators=100),method)
      Y_train = data["y_train"]
      compare_algorithms(X_train,Y_train.values.ravel(),X_test,y_test,method,dataset_object)

    if method == DR["XG"] and data_prediction_type == "Classification":
      X_test,X_train  = feature_selection(data,X_train_non_scaled,XGBClassifier(),method)
      Y_train = data["y_train"]
      compare_algorithms(X_train,Y_train.values.ravel(),X_test,y_test,method,dataset_object)
      
    # RFE is used with DecisionTreeClassifier and DecisionTreeRegressor for classificaion and regression respectively
    if method == DR["RFE"] and data_prediction_type == "Classification":
      n = dataset_object["FEATURES"]
      X_test,X_train  = recursive_fs(data,X_train_non_scaled,n,method)
      Y_train = data["y_train"]
      compare_algorithms(X_train,Y_train.values.ravel(),X_test,y_test,method,dataset_object)  
    
    if method == DR["RFE"] and data_prediction_type == "Regression":
      n = dataset_object["FEATURES"]      
      rfe = RFE(estimator=DecisionTreeRegressor(), n_features_to_select=n)
      model= DecisionTreeRegressor()
      pipeline = Pipeline(steps=[('s',rfe),('m',model)])
      pipeline = pipeline.fit(data["X_train"], data['y_train'])
      y_pred = pipeline.predict(data["X_test"])
      print("Results of:RFE\n")
      print('Mean Absolute Error:', metrics.mean_absolute_error(data["y_test"], y_pred))
      print('Mean Squared Error:', metrics.mean_squared_error(data["y_test"], y_pred))
      print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(data["y_test"], y_pred)))
      print(100*"=")
      # selector = selector.fit(X_train, y_train.values.ravel())
      # selected_feat = X_train_non_scaled.columns[(selector.get_support())]
      # print(f"Features Selected By {type}: {selected_feat}")
      # compare_algorithms(selector.transform(X_train), data["y_train"].ravel(),selector.transform(X_test), y_test,method,dataset_object)



      # # evaluate model
      # cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
      # n_scores = cross_val_score(pipeline,data["X_train"], data['y_train'], scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
      # # accuracy = cross_val_score(pipeline,data["X_train"], data['y_train'], cv=cv, scoring='accuracy', n_jobs=-1, error_score='raise')
      # print("Results for Type = Feature Selection | Model = RFE\n")
      # # print(f"Accuracy: {accuracy.mean()}")
      # print(f"MAE: {n_scores.mean()}")
      # print(100*"=")
      # # # report performance
      # print('MAE: %.3f (%.3f)' % (n_scores.mean()))
        

## **Applying DR/FS on Credit Card Dataset**

In [ ]:
def run_credit_card(n):
  CREDIT_CARD={
            "TYPE":"Classification",
            "N_COMPONENTS":n,
            "FEATURES":n,
            "DATAFRAME":cc_df,
            "ALGORITHMS":[ALGORITHMS["XGBOOST"],ALGORITHMS["RANDOM_FOREST"]]
            }
  start_comparison(CREDIT_CARD)

In [ ]:
run_credit_card(3)

Training Data Shape Before Applying FS or DR (15000, 23)
Training Data Shape After Applying PCA (15000, 3)
Results for Type = PCA | Model = XGBOOST
Accuracy: 0.8005333333333333
Precision: 0.617140313789666
ROC AUC: 0.7403855824631603


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metri

Results for Type = PCA | Model = RANDOM_FOREST
Accuracy: 0.7756666666666667
Precision: 0.11714285714285713
ROC AUC: 0.678271185242412
LDA:Getting Components
Best Number Of Components For LDA: 1
Results for Type = LDA | Model = XGBOOST
Accuracy: 0.8065999999999999
Precision: 0.6380687475893742
ROC AUC: 0.7145359095046062
Results for Type = LDA | Model = RANDOM_FOREST
Accuracy: 0.8087333333333333
Precision: 0.6569447109813501
ROC AUC: 0.7137858894219444
Features Selected By L1: Index(['PAY_0', 'PAY_2', 'BILL_AMT1', 'BILL_AMT3', 'BILL_AMT4', 'PAY_AMT1',
       'PAY_AMT2'],
      dtype='object')
Results for Type = L1 | Model = XGBOOST
Accuracy: 0.8150000000000001
Precision: 0.6804753274308576
ROC AUC: 0.7611134692570953
Results for Type = L1 | Model = RANDOM_FOREST
Accuracy: 0.8051333333333333
Precision: 0.7025281226498257
ROC AUC: 0.7499016251558379
Features Selected By RF: Index(['LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5',

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



Results for Type = RFE | Model = XGBOOST
Accuracy: 0.7748666666666666
Precision: 0.03333333333333333
ROC AUC: 0.5990929827789028


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metri

Results for Type = RFE | Model = RANDOM_FOREST
Accuracy: 0.7754666666666666
Precision: 0.0
ROC AUC: 0.5572727162502474


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
run_credit_card(10)

Training Data Shape Before Applying FS or DR (15000, 23)
Training Data Shape After Applying PCA (15000, 10)
Results for Type = PCA | Model = XGBOOST
Accuracy: 0.8002666666666667
Precision: 0.6208287226703955
ROC AUC: 0.7447024008645398
Results for Type = PCA | Model = RANDOM_FOREST
Accuracy: 0.7754666666666666
Precision: 0.0
ROC AUC: 0.6975459728035174
LDA:Getting Components
Best Number Of Components For LDA: 1
Results for Type = LDA | Model = XGBOOST
Accuracy: 0.8065999999999999
Precision: 0.6380687475893742
ROC AUC: 0.7145359095046062
Results for Type = LDA | Model = RANDOM_FOREST
Accuracy: 0.8087333333333333
Precision: 0.6569447109813501
ROC AUC: 0.7137858894219444
Features Selected By L1: Index(['PAY_0', 'PAY_2', 'BILL_AMT1', 'BILL_AMT3', 'BILL_AMT4', 'PAY_AMT1',
       'PAY_AMT2'],
      dtype='object')
Results for Type = L1 | Model = XGBOOST
Accuracy: 0.8150000000000001
Precision: 0.6804753274308576
ROC AUC: 0.7611134692570953
Results for Type = L1 | Model = RANDOM_FOREST
Accurac

In [ ]:
run_credit_card(15)

Training Data Shape Before Applying FS or DR (15000, 23)
Training Data Shape After Applying PCA (15000, 15)
Results for Type = PCA | Model = XGBOOST
Accuracy: 0.8088
Precision: 0.6429299679309481
ROC AUC: 0.757049849391391
Results for Type = PCA | Model = RANDOM_FOREST
Accuracy: 0.7754666666666666
Precision: 0.0
ROC AUC: 0.7121849924977665
LDA:Getting Components
Best Number Of Components For LDA: 1
Results for Type = LDA | Model = XGBOOST
Accuracy: 0.8065999999999999
Precision: 0.6380687475893742
ROC AUC: 0.7145359095046062
Results for Type = LDA | Model = RANDOM_FOREST
Accuracy: 0.8087333333333333
Precision: 0.6569447109813501
ROC AUC: 0.7137858894219444
Features Selected By L1: Index(['PAY_0', 'PAY_2', 'BILL_AMT1', 'BILL_AMT3', 'BILL_AMT4', 'PAY_AMT1',
       'PAY_AMT2'],
      dtype='object')
Results for Type = L1 | Model = XGBOOST
Accuracy: 0.8150000000000001
Precision: 0.6804753274308576
ROC AUC: 0.7611134692570953
Results for Type = L1 | Model = RANDOM_FOREST
Accuracy: 0.80513333

## **Applying DR/FS on Parkinson Dataset**

In [ ]:
def run_parkinson(n):
  PARKINSON_DISEASE={
            "TYPE":"Classification",
            "N_COMPONENTS":n,
            "FEATURES":n,
            "DATAFRAME":parkinson_df ,
            "ALGORITHMS":[ALGORITHMS["XGBOOST"],ALGORITHMS["RANDOM_FOREST"],ALGORITHMS["LOGISTIC_REGRESSION"]]
            }
  start_comparison(PARKINSON_DISEASE)

In [ ]:
run_parkinson(3)

Training Data Shape Before Applying FS or DR (378, 753)
Training Data Shape After Applying PCA (378, 3)
Results for Type = PCA | Model = XGBOOST
Accuracy: 0.7724039829302988
Precision: 0.8195921136653895
ROC AUC: 0.820650498926361
Results for Type = PCA | Model = RANDOM_FOREST
Accuracy: 0.7698435277382646
Precision: 0.7708939712248536
ROC AUC: 0.7769329800148765
LDA:Getting Components
Best Number Of Components For LDA: 1
Results for Type = LDA | Model = XGBOOST
Accuracy: 1.0
Precision: 1.0
ROC AUC: 1.0
Results for Type = LDA | Model = RANDOM_FOREST
Accuracy: 1.0
Precision: 1.0
ROC AUC: 1.0
Features Selected By L1: Index(['gender', 'DFA', 'RPDE', 'meanPeriodPulses', 'apq5Shimmer',
       'apq11Shimmer', 'f1', 'f2', 'b1', 'b2',
       ...
       'tqwt_kurtosisValue_dec_26', 'tqwt_kurtosisValue_dec_27',
       'tqwt_kurtosisValue_dec_28', 'tqwt_kurtosisValue_dec_30',
       'tqwt_kurtosisValue_dec_31', 'tqwt_kurtosisValue_dec_32',
       'tqwt_kurtosisValue_dec_33', 'tqwt_kurtosisValue_de

In [ ]:
run_parkinson(10)

Training Data Shape Before Applying FS or DR (378, 753)
Training Data Shape After Applying PCA (378, 10)
Results for Type = PCA | Model = XGBOOST
Accuracy: 0.7933854907539118
Precision: 0.8233446995870931
ROC AUC: 0.8270110030454857
Results for Type = PCA | Model = RANDOM_FOREST
Accuracy: 0.7514224751066856
Precision: 0.7498080654737002
ROC AUC: 0.7883521395590362
LDA:Getting Components
Best Number Of Components For LDA: 1
Results for Type = LDA | Model = XGBOOST
Accuracy: 1.0
Precision: 1.0
ROC AUC: 1.0
Results for Type = LDA | Model = RANDOM_FOREST
Accuracy: 1.0
Precision: 1.0
ROC AUC: 1.0
Features Selected By L1: Index(['gender', 'DFA', 'RPDE', 'meanPeriodPulses', 'apq5Shimmer',
       'apq11Shimmer', 'f1', 'f2', 'b1', 'b2',
       ...
       'tqwt_kurtosisValue_dec_26', 'tqwt_kurtosisValue_dec_27',
       'tqwt_kurtosisValue_dec_28', 'tqwt_kurtosisValue_dec_30',
       'tqwt_kurtosisValue_dec_31', 'tqwt_kurtosisValue_dec_32',
       'tqwt_kurtosisValue_dec_33', 'tqwt_kurtosisValue_

## **Applying DR/FS on Super Conduct Dataset**

In [ ]:
def run_superconduct(n):
  SuperConduct={
            "TYPE":"Regression",
            "N_COMPONENTS":n,
            "FEATURES":n,
            "DATAFRAME":sc_df ,
            "ALGORITHMS":[ALGORITHMS["LINEAR_REGRESSION"],ALGORITHMS["VOTING_REGRESSION"]]
            }
  start_comparison(SuperConduct)

In [ ]:
run_superconduct(5)

Training Data Shape Before Applying FS or DR (10631, 81)
Training Data Shape After Applying PCA (10631, 5)
Results of:LINEAR_REGRESSION
Mean Absolute Error: 18.51433052672795
Mean Squared Error: 544.2788952707141
Root Mean Squared Error: 23.32978558132745
Results of:VOTING_REGRESSION
Mean Absolute Error: 11.882004243276619
Mean Squared Error: 254.9286719615991
Root Mean Squared Error: 15.966485898957199
Results of:RFE

Mean Absolute Error: 7.558774239901734
Mean Squared Error: 210.2513158775635
Root Mean Squared Error: 14.5000453750174


In [ ]:
run_superconduct(10)

Training Data Shape Before Applying FS or DR (10631, 81)
Training Data Shape After Applying PCA (10631, 10)
Results of:LINEAR_REGRESSION
Mean Absolute Error: 17.794620110803294
Mean Squared Error: 500.7630304242782
Root Mean Squared Error: 22.3777351495695
Results of:VOTING_REGRESSION
Mean Absolute Error: 11.301110551000908
Mean Squared Error: 226.08934359331764
Root Mean Squared Error: 15.036267608463133
Results of:RFE

Mean Absolute Error: 7.155985477663248
Mean Squared Error: 189.35928012930768
Root Mean Squared Error: 13.760787772845989


In [ ]:
run_superconduct(20)

Training Data Shape Before Applying FS or DR (10631, 81)
Training Data Shape After Applying PCA (10631, 20)
Results of:LINEAR_REGRESSION
Mean Absolute Error: 16.674840792234455
Mean Squared Error: 438.6716242609939
Root Mean Squared Error: 20.944489114346855
Results of:VOTING_REGRESSION
Mean Absolute Error: 10.668648866685283
Mean Squared Error: 205.4018988111522
Root Mean Squared Error: 14.331849106488395
Results of:RFE

Mean Absolute Error: 7.021821420217225
Mean Squared Error: 186.99307785565603
Root Mean Squared Error: 13.674541230171345


## **Applying DR/FS on Mice Protein Dataset**

In [9]:
def run_mice(n):
  mice_protein={
            "TYPE":"Regression",
            "N_COMPONENTS":n,
             "FEATURES":n,
            "DATAFRAME":mice_df ,
            "ALGORITHMS":[ALGORITHMS["LINEAR_REGRESSION"],ALGORITHMS["VOTING_REGRESSION"]]
            }
  start_comparison(mice_protein)

In [ ]:
run_mice(5)

Training Data Shape Before Applying FS or DR (276, 79)
Training Data Shape After Applying PCA (276, 5)
Results of:LINEAR_REGRESSION
Mean Absolute Error: 0.20609235531272235
Mean Squared Error: 0.06448774892562276
Root Mean Squared Error: 0.25394438155947213
Results of:VOTING_REGRESSION
Mean Absolute Error: 0.1316465568713598
Mean Squared Error: 0.03485491131083742
Root Mean Squared Error: 0.18669470081080883
Results of:RFE

Mean Absolute Error: 0.0036231884057971015
Mean Squared Error: 0.0036231884057971015
Root Mean Squared Error: 0.0601929265428846


In [ ]:
run_mice(12)

Training Data Shape Before Applying FS or DR (276, 79)
Training Data Shape After Applying PCA (276, 12)
Results of:LINEAR_REGRESSION
Mean Absolute Error: 0.13769128917446471
Mean Squared Error: 0.03361517651365095
Root Mean Squared Error: 0.18334442045955734
Results of:VOTING_REGRESSION
Mean Absolute Error: 0.10371526897978973
Mean Squared Error: 0.02552141105897557
Root Mean Squared Error: 0.15975422078610496
Results of:RFE

Mean Absolute Error: 0.0036231884057971015
Mean Squared Error: 0.0036231884057971015
Root Mean Squared Error: 0.0601929265428846


In [ ]:
run_mice(20)

Training Data Shape Before Applying FS or DR (276, 79)
Training Data Shape After Applying PCA (276, 20)
Results of:LINEAR_REGRESSION
Mean Absolute Error: 0.1029971788913383
Mean Squared Error: 0.016341728620962316
Root Mean Squared Error: 0.1278347707822966
Results of:VOTING_REGRESSION
Mean Absolute Error: 0.07821925515157233
Mean Squared Error: 0.015133624726938888
Root Mean Squared Error: 0.12301879826652058
Results of:RFE

Mean Absolute Error: 0.0036231884057971015
Mean Squared Error: 0.0036231884057971015
Root Mean Squared Error: 0.0601929265428846
